In [23]:
import torch
import torchvision.models as models
from torch.profiler import profile, record_function, ProfilerActivity


In [39]:
%env MKLDNN_VERBOSE=1
%env DNNL_VERBOSE=1
%env ONEDNN_VERBOSE=1

env: MKLDNN_VERBOSE=1
env: DNNL_VERBOSE=1
env: ONEDNN_VERBOSE=1


In [32]:
model = models.resnet18()
inputs = torch.randn(5, 3, 224, 224)

In [33]:
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
        model(inputs)

onednn_verbose,info,oneDNN v2.7.1 (commit N/A)
onednn_verbose,info,cpu,runtime:threadpool,nthr:6
onednn_verbose,info,cpu,isa:Intel AVX2
onednn_verbose,info,gpu,runtime:none
onednn_verbose,info,prim_template:operation,engine,primitive,implementation,prop_kind,memory_descriptors,attributes,auxiliary,problem_desc,exec_time
onednn_verbose,exec,cpu,gemm_api,,undef,src_f32::blocked:ab:f0 wei_f32::blocked:ab:f0 dst_f32::blocked:ab:f0,,3x4:4x3,15.8098
onednn_verbose,exec,cpu,gemm_api,,undef,src_f32::blocked:ab:f0 wei_f32::blocked:ab:f0 dst_f32::blocked:ab:f0,,3x4:4x3,0.0078125


STAGE:2023-02-14 12:20:58 2949489:2949489 ActivityProfilerController.cpp:294] Completed Stage: Warm Up
STAGE:2023-02-14 12:20:58 2949489:2949489 ActivityProfilerController.cpp:300] Completed Stage: Collection


In [17]:
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                  model_inference        -2.83%   -2724.000us        99.98%      96.264ms      96.264ms             1  
                     aten::conv2d         4.78%       4.599ms        71.63%      68.964ms       3.448ms            20  
                aten::convolution         0.26%     247.000us        71.54%      68.876ms       3.444ms            20  
               aten::_convolution         0.16%     157.000us        71.28%      68.629ms       3.431ms            20  
         aten::mkldnn_convolution        70.96%      68.317ms        71.12%      68.472ms       3.424ms            20  
                 aten::batch_norm       

In [19]:
print(prof.key_averages(group_by_input_shape=True).table(sort_by="cpu_time_total", row_limit=10))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls                                                                      Input Shapes  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  
                  model_inference        -2.83%   -2724.000us        99.98%      96.264ms      96.264ms             1                                                                                []  
                     aten::conv2d         0.01%      11.000us        17.56%      16.906ms       5.635ms             3                            [[5, 512, 7, 7], [512, 512, 3, 3], [], [], [], 

In [22]:
model = models.resnet18()
inputs = torch.randn(5, 3, 224, 224)

with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA]) as prof:
    model(inputs)

prof.export_chrome_trace("trace.json")

/home/taosy/.local/lib/python3.9/site-packages/torch/autograd/profiler.py:176: UserWarning: CUDA is not available, disabling CUDA profiling
  warn("CUDA is not available, disabling CUDA profiling")
STAGE:2023-02-13 02:11:38 2949489:2949489 ActivityProfilerController.cpp:294] Completed Stage: Warm Up
STAGE:2023-02-13 02:11:38 2949489:2949489 ActivityProfilerController.cpp:300] Completed Stage: Collection
ERROR:2023-02-13 02:11:38 2949489:2949489 CudaDeviceProperties.cpp:26] cudaGetDeviceCount failed with code 35
STAGE:2023-02-13 02:11:38 2949489:2949489 output_json.cpp:417] Completed Stage: Post Processing


In [25]:
p=torch.range(0,24)

/tmp/ipykernel_2949489/2013822537.py:1: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  p=torch.range(0,24)


In [27]:
import tensorflow as tf

2023-02-14 12:18:05.886344: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-14 12:18:06.543522: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-14 12:18:06.543590: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-02-14 12:18:06.543600: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [34]:
data=[[1.0,2.0,4.0,5.0],[0.0,6.0,7.0,8.0],[8.0,1.0,1.0,1.0]]
X=tf.constant(data)
matResult=tf.matmul(X, X, transpose_b=True)
print(matResult)

tf.Tensor(
[[ 46.  80.  19.]
 [ 80. 149.  21.]
 [ 19.  21.  67.]], shape=(3, 3), dtype=float32)


In [40]:
m = torch.nn.Linear(20, 30)
input = torch.randn(128, 20)
output = m(input)
print(output.size())

torch.Size([128, 30])
